In [1]:
import numpy as np
import pandas as pd
import os
import tqdm

import sys
sys.path.append('../')
import itertools

from util.text_preprocessing import tokenize_comment

# Load Data

## 전체 코멘트 불러오기

In [2]:
total_comments_df = pd.read_pickle("dataset/total-comments_tokenized.pkl")
total_comments_df = total_comments_df.loc[:,['comment_id','comment','comment_token']]
total_comments_df.head(5)

,comment_id,comment,comment_token
0,F3C2DFCFE9E947B19C8A3053DD5C821B,이국종교수는 외과 야전사령관 이다. 그분의 업무에 차질 없도록 물심양면 으로 ...,"[이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없도록, 물, 심양..."
1,61A3B6AAD6124DFFB70EC4275496F8A6,"문재인 대통령님과 이국종 교수님, 그리고 귀순병사 모두 건강하시길 빕니다.","[문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하시길]"
2,B6660EF2C7C14A2F903A08363641CF09,단 한번도 써보지 못했다ㆍ 포인트의 절반만이라도 통신비로 차감을!,"[단, 한번, 보지, 포인트, 절반, 통신비, 차감]"
3,C4F129BFB1FD4543A24A03D96F38351D,우리나라 통신업체들 참 쉽게 돈 벌어요,"[우리나라, 통신, 업체, 돈]"
4,1A21FDDF151E45DF808F0E5B662E8F3C,진짜 쓸데가너무적다.,[진짜]


## 감정 사전 만들기
- 6 emotion : happy, sad, disgust, angry, surprised, fear
- 홍종선, 정연주. (2009). 감정동사의 범주 규정과 유형 분류. 한국어학, 45(), 387-420.

In [3]:
six_emodict_df = pd.read_excel("자료/six_emotion_bigram2_0403.xlsx", encoding = 'utf-8')
six_emodict_df.head()

,happy,sad,disgust,angry,surprised,fear
0,가뿐하다,가슴앓이,가소롭다,갈기갈기,갑작스럽다,가혹하다
1,감개무량하다,가엽다,거북하다,개새끼,경악하다,강압적
2,감격스럽다,가엾다,경박하다,개자식,경이,겁쟁이
3,감격하다,가혹하다,괴상하다,격노하다,급작스럽다,공포감
4,감동스럽다,각박하다,괴팍하다,격분하다,기겁하다,공포스럽다


In [4]:
# 감정 별 단어 개수
six_emodict_df.count()

happy        113
sad          174
disgust       77
angry        110
surprised     47
fear          38
dtype: int64

In [5]:
# 감정 리스트에 변수 할당 
six_emotion = six_emodict_df.columns.tolist()
print(six_emotion)
for emo in six_emotion:
    # 단어 사전에서 emo_words 받아오기
    emo_words = six_emodict_df[emo].dropna().tolist() 
    # 단어 토큰화, 토큰화 후 nan값 된 단어 제거
    emo_words = [tokenize_comment(x)[0] for x in emo_words if len(tokenize_comment(x))>0]
    # 중복 제거
    emo_words = sorted(list(set(emo_words)))
    # 각 감정변수에 차례대로 할당
    globals()[emo] = emo_words
    
angry[:5]

['happy', 'sad', 'disgust', 'angry', 'surprised', 'fear']


['가당찮다', '가증', '갈기갈기', '갈등', '강압']

In [6]:
bi_happy = [('속','시원하다'),('낯','간지럽다'),('정','들다'),('홀','가분')]

bi_sad = [('가슴','아프다'),('기운','없다'),('남','부럽다'),('뒤','숭숭'),('맥','빠지다'),
              ('복','받치다'),('뼈','아프다'),('뼈','저리다'),('애','끓다'),('애','닮다'),
              ('애','타다'),('처','연하다'),('속','썩다'),('속','앓이'),('속','타다')]

bi_disgust =  [('못','밉다'),('정','떨어지다'),('못','마땅하다'),('시','덥다'),('남','부끄럽다'),
                   ('징','그렇다'),('뼈','저리다'),('애','끓다'),('애','닮다'),('처','연하다'),
                   ('속','썩다'),('속','앓이'),('속','타다')]

bi_angry = [('못','되다'),('삐','딱하다'),('비','딱하다'),('약','오르다'),('치','떨리다'),
                ('치밀','오르다'),('성','나다'),('못','마땅하다'),('정','떨어지다'),('남','부끄럽다')]

bi_surprised = [('기상','천외'),('얼','떨다')]

bi_fear = [('뒤','숭숭')]

# 코멘트 데이터에 감정 레이블 달기 

In [7]:
test = ["가슴 아프다 정말", #- sad
        "요즘 상황이 뒤숭숭하다",#sad fear
        "어어어어엉 우웅우 감동스럽다"]#happy]
        
test_token = []
for i in test:
    test_token.append(tokenize_comment(i))
print(test_token)

[['가슴', '아프다', '정말'], ['요즘', '상황', '뒤', '숭숭'], ['감동', '스럽다']]


## 댓글에 특정 감정에 해당하는 단어가 등장하면 update

- bigram
- unigram

In [89]:
total_comments_df['comment'][:10]

0    이국종교수는  외과 야전사령관 이다.  그분의 업무에 차질 없도록 물심양면 으로  ...
1            문재인 대통령님과 이국종 교수님, 그리고 귀순병사 모두 건강하시길 빕니다.
2                 단 한번도 써보지 못했다ㆍ 포인트의 절반만이라도 통신비로 차감을!
3                              우리나라 통신업체들  참 쉽게  돈 벌어요
4                                          진짜 쓸데가너무적다.
5                                포인트 다쓰려다 내돈이 더 많이 나간다
6                                  사법 적폐 청산이 시급한듯 싶네요!
7       죽고자하면산다고 죽음에사선을넘어온  그대가  대한민국에세 아름다운삶을이어가기바랍니다
8    당시 우리국군과 미군들의  대응은 아주 잘한것으로 보입니다, 공동경비 구역의 모든 ...
9                             우리 국방부 대응 잘했어요?? 문제는 언론임
Name: comment, dtype: object

In [96]:
from konlpy.tag import Okt
def tokenize_comment(comment):
    """ 
    ** 참고 - konlpy.Okt 패키지 사용 **
    
    :param comment: 뉴스 댓글 문장 하나
    :return: 문장을 token단위로 쪼갠 후 리스트로 반환
    
    """
    okt = Okt()
    i = 0
    malist = okt.pos(comment)
    r = []

    tag = ["Noun", "Adjective"]
    stopwords = ["것", "이", "안", "더", "왜", "때", "좀", "뭐", "거", "저", "뿐", "머"]

    try:
        for word in malist:
            # 어미/조사/구두점/ㅋㅋ^^ㅎㅎ/음표살림/Alphabet/부사는 대상에서 제외
            if word[1] in tag:
                if not word[0] in r:
                    if not word[0] in stopwords:
                        # 숫자, 특수문자 제거.
                        r.append(word[0])
        return r

    except Exception as e:
        print(e)

In [97]:
tokenize_comment("사법 적폐 청산이 시급한듯 싶네요!")

['사법', '적폐', '청산', '시급한듯']

2393070

In [117]:
emo_set = [happy, sad, disgust, angry, surprised, fear]
emo_bigram_set = [bi_happy, bi_sad, bi_disgust, bi_angry, bi_surprised, bi_fear]

check_token_lst = []
emo_cnt_lst = []

total_num = len(total_comments_df)

# comment 돌기
for i,comment in enumerate(total_comments_df['comment_token']):
    print(i/total_num,end="\r")
    check_token = []
    emo_cnt = [0,0,0,0,0,0]
    
    # bigram 처리
    for i, biwords in enumerate(emo_bigram_set):
        for a, b in biwords:
             if (a in comment) & (b in comment):
                check_token.append(a+b)
                emo_cnt[i] += 1 

    # unigram 처리
    for token in comment:
        for i, words in enumerate(emo_set):
            if token in words:
                emo_cnt[i] += 1
                check_token.append(token)

    check_token_lst.append(check_token)
    emo_cnt_lst.append(emo_cnt)

In [151]:
# 비율로 환산
sum_of_emo = np.array(emo_cnt_lst).sum(axis=1,keepdims=True)

emo_ratio = np.array(emo_cnt_lst) / sum_of_emo
emo_ratio[np.isnan(emo_ratio)] = 0

emo_df = pd.DataFrame(emo_ratio,columns = ['happy','sad','disgust','angry','surprised','fear'])

In [155]:
a = emo_df['happy']+emo_df['sad']+emo_df['disgust']+emo_df['angry']+emo_df['surprised']+emo_df['fear']
len(np.where(np.array(a)>0)[0])

400917

2,393,070 개의 댓글 중 40만 개의 댓글이 감정 단어를 보유

.... 6개 중 1개 꼴

In [156]:
comment_with_emo = pd.concat([emo_df,
                              total_comments_df,
                              pd.DataFrame({"check":pd.Series(check_token_lst)})],
                             sort=False,axis=1)
print(comment_with_emo.shape)
comment_with_emo.head()

(2393070, 10)


,happy,sad,disgust,angry,surprised,fear,comment_id,comment,comment_token,check
0,0.0,0.0,0.0,0.0,0.0,0.0,F3C2DFCFE9E947B19C8A3053DD5C821B,이국종교수는 외과 야전사령관 이다. 그분의 업무에 차질 없도록 물심양면 으로 ...,"[이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없도록, 물, 심양...",[]
1,0.0,0.0,0.0,0.0,0.0,0.0,61A3B6AAD6124DFFB70EC4275496F8A6,"문재인 대통령님과 이국종 교수님, 그리고 귀순병사 모두 건강하시길 빕니다.","[문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하시길]",[]
2,0.0,0.0,0.0,0.0,0.0,0.0,B6660EF2C7C14A2F903A08363641CF09,단 한번도 써보지 못했다ㆍ 포인트의 절반만이라도 통신비로 차감을!,"[단, 한번, 보지, 포인트, 절반, 통신비, 차감]",[]
3,0.0,0.0,0.0,0.0,0.0,0.0,C4F129BFB1FD4543A24A03D96F38351D,우리나라 통신업체들 참 쉽게 돈 벌어요,"[우리나라, 통신, 업체, 돈]",[]
4,0.0,0.0,0.0,0.0,0.0,0.0,1A21FDDF151E45DF808F0E5B662E8F3C,진짜 쓸데가너무적다.,[진짜],[]
